In [1]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir)))


import DifferentiationInterface.src.diffipy.diffipy as dp

2024-06-16 17:17:22.359858: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-16 17:17:22.359880: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-16 17:17:22.360441: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-16 17:17:22.765554: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
backend_array = ['numpy', 'torch',  'tensorflow', 'jax', 'aadc'] # aadc not pre-computing tape yet...

for backend in backend_array:
    dp.setBackend(backend)
    # Initialize variables and constants
    s0_value = 100.0
    K_value = 110.0
    r_value = 0.05
    sigma_value = 0.3
    dt_value = 1.0
    z_value = 0.5
    N = 1000000
    seed = 1

    # Define variables
    s0 = dp.variable(s0_value, 'input','s0')
    K = dp.variable(K_value, 'input','K')
    r = dp.variable(r_value, 'input','r')
    sigma = dp.variable(sigma_value, 'input','sigma')
    dt = dp.variable(dt_value, 'input','dt')
    z = dp.variable(z_value, 'randomVariableNormal','z')

    func_input_variables = [s0, K, r, sigma, dt, z]
    diff_variables = [s0, K, r, sigma, dt]

    # Record Tape: Standard Monte Carlo

    s = s0 * dp.exp((r - sigma **2 / 2) * dt + sigma * dp.sqrt(dt) * z)
    payoff =  dp.if_(s > K, s - K, 0)
    PV_standard = dp.exp(-r * dt) * dp.sum(payoff) / N

    ### Test performance # seed and NewSample not working for aadc yet
    #dp.seed(seed)
    #pre_computed_random_variables = z.NewSample(N) # pre-evaluate random samples

    PV_standard.run_performance_test(func_input_variables, diff_variables, warmup_iterations = 3, test_iterations = 10)


Backend              Eval-Result  mean runtime    variance runtime    gradient directions: {'r', 'sigma', 'K', 's0', 'dt'}
numpy                0.000006     0.000151        0.000000            [1.1107106110097272e-06, -9.512294248857849e-07, 0.00010463471352264372, 2.2213679065131473e-05, 8.563872032783175e-06]
numpy_as_func        0.000006     0.000003        0.000000            {'s0': 1.1107106110097272e-06, 'K': -9.512294248857849e-07, 'r': 0.00010463471352264372, 'sigma': 2.2213679065131473e-05, 'dt': 8.563872032783175e-06}
torch                0.000006     0.000713        0.000000            [1.1107105137853068e-06, -9.512293672742089e-07, 0.00010463522630743682, 2.221421163994819e-05, 8.563893970858771e-06]
torch_as_func        0.000006     0.000127        0.000000            [tensor(1.1107e-06), tensor(-9.5123e-07), tensor(0.0001), tensor(2.2214e-05), tensor(8.5639e-06)]
tensorflow           0.000006     0.006507        0.000000            [1.1107106e-06, -9.512294e-07, 0.000104